In [26]:
import os;
import time;
from selenium import webdriver;
from selenium.webdriver import ActionChains

# concat images
from PIL import Image;
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM

import base64;
category = "";
dir = "D:/Documents/dev/python/photomap/todo/";
# dir = "D:/Documents/dev/python/photomap/data/"; category = "M50/tmp";

download = "D:/Downloads/download.svg";

dirsvg = "svg/";
dircollage = "collage/";
extensionphoto = ".jpg";

In [27]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def saveSVG(name):
    file = dir + category + name + extensionphoto;
    
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
#     chrome_options.add_experimental_option("prefs",prefs)
#     chrome_options.add_argument('--disable-notifications')
#     chrome_options.add_argument('--disable-alerts')

    driver = webdriver.Chrome("./chromedriver.exe", options=chrome_options)

    ############# SVG
    driver.get("http://www.geotests.net/couleurs/index.html");
    
    chooseFile = driver.find_element_by_id("files");
    chooseFile.send_keys(file);

#     driver.switch_to.alert.accept();
    
    try:
        WebDriverWait(driver, 5).until (EC.alert_is_present())
        alert = driver.switch_to.alert
        alert.accept()
    except:
        pass
    try:
        WebDriverWait(driver, 5).until (EC.alert_is_present())
        alert = driver.switch_to.alert
        alert.accept()
    except:
        pass
#     try:
#         driver.switch_to.alert.accept();
#     except:
#         pass

    export = driver.find_element_by_id("svgSVG");
    ActionChains(driver).click(export).perform();

    time.sleep(5); # Wait DL

    png = driver.find_element_by_id("canvasImg");

    # save to a file
    with open(download, 'rb') as f:
        with open(dirsvg + category + name + ".svg", 'wb') as g:
            g.write(f.read());
            
    os.remove(download);
    driver.quit()

In [33]:
with os.scandir(dir + category) as entries:
    for entry in entries:
        saveSVG(entry.name[0:-4]);

In [36]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, int(abs(im1.height - im2.height) / 2)))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

with os.scandir(dir + category) as entries:
    for entry in entries:
        name = entry.name[0:-4];
        drawing = svg2rlg(dirsvg + category + name + ".svg");
        drawing.scale(5,5);
        renderPM.drawToFile(drawing, name + ".png", "PNG",360); # write map

        im1 = Image.open(dir + category + name + extensionphoto);
        im2 = Image.open(name + ".png");

        get_concat_h(im1, im2).save(dircollage + category + name + ".jpg")
        # get_concat_v(im1, im2).save("concat_v.jpg")
        os.remove(dir + category + name + extensionphoto);
        os.remove(name +".png");